# **SETUP**

First we need to install DuckDB.


In [1]:
!pip install duckdb --pre

     |████████████████████████████████| 14.0 MB 4.9 MB/s 


# **Loading The Data**


We will work with a generated dataset from the TPC-H benchmark. DuckDB has built-in support for generating the dataset using the `dbgen` procedure.

We create an in-memory database and generate the data inside DuckDB using the following code snippet.



In [2]:
import duckdb
con = duckdb.connect(':memory:')
con.execute("CALL dbgen(sf=0.1)")

# **Inspecting the Dataset**

The dataset consists of eight tables. We can see which tables are present in the database using the `SHOW TABLES` command.

Note that we append `.df()` to the query, this fetches the result as a Pandas DataFrame which renders nicely in Colab.

In [3]:
con.execute("SHOW TABLES").df()

,name
0,customer
1,lineitem
2,nation
3,orders
4,part
5,partsupp
6,region
7,supplier


Using the `DESCRIBE` command, we can inspect the columns that are present in each of the tables. For example, we can inspect the lineitem table as follows:

In [5]:
con.execute("DESCRIBE lineitem").df()

,column_name,column_type,null,key,default,extra
0,l_orderkey,INTEGER,NO,NaN,NaN,NaN
1,l_partkey,INTEGER,NO,NaN,NaN,NaN
2,l_suppkey,INTEGER,NO,NaN,NaN,NaN
3,l_linenumber,INTEGER,NO,NaN,NaN,NaN
4,l_quantity,INTEGER,NO,NaN,NaN,NaN
5,l_extendedprice,"DECIMAL(15,2)",NO,NaN,NaN,NaN
6,l_discount,"DECIMAL(15,2)",NO,NaN,NaN,NaN
7,l_tax,"DECIMAL(15,2)",NO,NaN,NaN,NaN
8,l_returnflag,VARCHAR,NO,NaN,NaN,NaN
9,l_linestatus,VARCHAR,NO,NaN,NaN,NaN


We can use the `LIMIT` clause to inspect the first few rows of the lineitem table and display them.

In [6]:
con.execute("SELECT * FROM lineitem LIMIT 10").df()

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment
0,1,15519,785,1,17,24386.67,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,egular courts above the
1,1,6731,732,2,36,58958.28,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,ly final dependencies: slyly bold
2,1,6370,371,3,8,10210.96,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,"riously. regular, express dep"
3,1,214,465,4,28,31197.88,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,lites. fluffily even de
4,1,2403,160,5,24,31329.60,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,pending foxes. slyly re
5,1,1564,67,6,32,46897.92,0.07,0.02,N,O,1996-01-30,1996-02-07,1996-02-03,DELIVER IN PERSON,MAIL,arefully slyly ex
6,2,10617,138,1,38,58049.18,0.00,0.05,N,O,1997-01-28,1997-01-14,1997-02-02,TAKE BACK RETURN,RAIL,ven requests. deposits breach a
7,3,430,181,1,45,59869.35,0.06,0.00,R,F,1994-02-02,1994-01-04,1994-02-23,NONE,AIR,ongside of the furiously brave acco
8,3,1904,658,2,49,88489.10,0.10,0.00,R,F,1993-11-09,1993-12-20,1993-11-24,TAKE BACK RETURN,RAIL,unusual accounts. eve
9,3,12845,370,3,27,47461.68,0.06,0.07,A,F,1994-01-16,1993-11-22,1994-01-23,DELIVER IN PERSON,SHIP,nal foxes wake.


To get a better feeling of what a table contains, we can use the `SUMMARIZE` command. This prints out several statistics about each of the columns of the table, such as the min and max value, how many unique values there are, the average value in the column, etc.

In [14]:
con.execute("SUMMARIZE lineitem").df()
# con.execute("SUMMARIZE lineitem").fetchone()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,l_orderkey,INTEGER,1,600000,152234,300087.32032628893,173380.15954532538,151126,299763,450987,600572,0.0%
1,l_partkey,INTEGER,1,20000,20024,10003.995747387491,5768.962517282611,5014,10006,14995,600572,0.0%
2,l_suppkey,INTEGER,1,1000,1005,500.55533391500103,288.5557626111953,250,500,749,600572,0.0%
3,l_linenumber,INTEGER,1,7,7,3.001215507882485,1.7321171926128476,1,3,4,600572,0.0%
4,l_quantity,INTEGER,1,50,50,25.53366124294839,14.420188609142347,13,25,37,600572,0.0%
5,l_extendedprice,"DECIMAL(15,2)",901.00,95949.50,132041,35992.236201887536,22014.18174976392,17619,34465,51656,600572,0.0%
6,l_discount,"DECIMAL(15,2)",0.00,0.10,11,0.050073929520523766,0.031603128191052286,0,0,0,600572,0.0%
7,l_tax,"DECIMAL(15,2)",0.00,0.08,9,0.040041626982276896,0.025797304713675414,0,0,0,600572,0.0%
8,l_returnflag,VARCHAR,A,R,3,NaN,NaN,NaN,NaN,NaN,600572,0.0%
9,l_linestatus,VARCHAR,F,O,2,NaN,NaN,NaN,NaN,NaN,600572,0.0%


# **Testing and Benchmarking**

Let us start our assignment by running a microbenchmark against the TPC-H dataset.

In order to make the benchmarking more interesting, let's compare against the SQLite database. This is a typical OLTP (transactional) database that is included along with every Python installation.


### **SQLite Setup**

We start out by creating a new in-memory database, just as we did in DuckDB.

In [23]:
import duckdb
con = duckdb.connect(':memory:')
con.execute("CALL dbgen(sf=0.1)")

import sqlite3
sqlite_con = sqlite3.connect(':memory:', check_same_thread=False)

We can transfer the data from DuckDB to SQLite using a Pandas DataFrame as well. First, we export the data from DuckDB into a Pandas DataFrame using the `.df()` command. Then we use the `to_sql` function to write the data to our SQLite database.

In [28]:
%%time
table_list = con.execute("SHOW TABLES").fetchall()

for table in table_list:
  tname = table[0]
  table_data = con.table(tname).df()
  table_data.to_sql(tname, sqlite_con, if_exists="replace")

CPU times: user 10 s, sys: 398 ms, total: 10.4 s
Wall time: 10.4 s


In [ ]:
%%time
table_list = con.execute("SHOW TABLES").fetchall()

[con.table(table[0]).df().to_sql(tname, sqlite_con, if_exists="replace") for table[0] in table_list]


In [25]:
table_list
# sqlite_con.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

[('customer',),
 ('lineitem',),
 ('nation',),
 ('orders',),
 ('part',),
 ('partsupp',),
 ('region',),
 ('supplier',)]

# **Running the Benchmark**

We have created a query down below which resembles a (simplified) query from the TPC-H benchmark:

In [26]:
query = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 10;
"""

Let's run the query in both SQLite and in DuckDB and measure the execution time.

In [29]:
import time
import pandas as pd


def run_query(con_obj, q):
  start = time.time()
  con_obj.execute(q).fetchall()
  end = time.time()
  return(str(round(end - start,3)) + 's')

duckdb_results = [run_query(con, query)]
sqlite_results = [run_query(sqlite_con, query)]

pd.DataFrame.from_dict({
    'DuckDB': duckdb_results,
    'SQLite': sqlite_results
})

,DuckDB,SQLite
0,0.028s,0.414s


Using the `PRAGMA disable_optimizer` we can also disable the query optimizer of DuckDB, and re-run the query. In this manner we can see the performance effect that query optimization has on our query.

In [34]:
con.execute("PRAGMA disable_optimizer")
duckdb_unoptimized_results = [run_query(con, query)]
con.execute("PRAGMA enable_optimizer")

pd.DataFrame.from_dict({
    'DuckDB': duckdb_results,
    'DuckDB (Unoptimized)': duckdb_unoptimized_results,
    'SQLite': sqlite_results
})

,DuckDB,DuckDB (Unoptimized),SQLite
0,0.028s,0.097s,0.414s


# **Inspecting the Query Plan**
The query plan of a query can be inspected by prefixing the query with`EXPLAIN`. By default, only the physical query plan is returned. You can use `PRAGMA explain_output='all'` to output the unoptimized logical plan, the optimized logical plan and the physical plan as well.

In [ ]:
def explain_query(query):
  print(con.execute("EXPLAIN " + query).fetchall()[0][1])

explain_query(query)

# **Profiling Queries**
Rather than only viewing the query plan, we can also run the query and look at the profile output. The function `run_and_profile_query` below performs this profiling by enabling the profiling, writing the profiling output to a file, and then printing the contents of that file to the console.

The profiler output shows extra information for every operator; namely how much time was spent executing that operator, and how many tuples have moved from that operator to the operator above it. 

For a `SEQ_SCAN` (sequential scan), for example, it shows how many tuples have been read from the base table. For a `FILTER`, it shows how many tuples have passed the filter predicate. For a``HASH_GROUP_BY`, it shows how many groups were created and aggregated.

These intermediate cardinalities are important because they do a good job of explaining why an operator takes a certain amount of time, and in many cases these intermediates can be avoided or drastically reduced by modifying the way in which a query is executed.



In [ ]:
def run_and_profile_query(query):
  con.execute("PRAGMA enable_profiling")
  con.execute("PRAGMA profiling_output='out.log'")
  con.execute(query)
  con.execute("PRAGMA disable_profiling")
  with open('out.log', 'r') as f:
    output = f.read()
  print(output)
  
run_and_profile_query(query)


# **Query Optimizations**

An important component of a database system is the optimizer. The optimizer changes the query plan so that it is logically equivalent to the original plan, but (hopefully) executes much faster.

In an ideal world, the optimizer allows the user not to worry about how to formulate a query: the user only needs to describe what result they want to see, and the database figures out the most efficient way of retrieving that result.

In practice, this is certainly not always true, and in some situations it is necessary to rephrase a query. Nevertheless, optimizers generally do a very good job at optimizing queries, and save users a lot of time in manually reformulating queries.

Let us run the following query and see how it performs:

In [ ]:
unoptimized_query = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer,
    orders,
    lineitem
WHERE
    c_mktsegment = 'BUILDING'
    AND c_custkey = o_custkey
    AND l_orderkey = o_orderkey
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 10;
"""

run_and_profile_query(unoptimized_query)




# **Manual Query Optimizations**

In order to get a better idea of how query optimizers work, we are going to perform *manual* query optimization. In order to do that, we will disable all query optimizers in DuckDB, which means the query will run *as-is*. We can then change the way the query is physically executed by altering the query. Let's try to disable the optimizer and looking at the query plan:



In [ ]:
# con.execute("PRAGMA disable_optimizer")
explain_query(unoptimized_query)


Looking at the plan you now see that the hash joins that were used before are replaced by cross products followed by a filter. This is what was literally written in the query, however, cross products are extremely expensive! We could run this query, but because of the cross products it will take extremely long. 

Let's rewrite the query to explicitly use joins instead, and then we can actually run it:

In [ ]:
query = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 10;
"""

run_and_profile_query(query)


# **Assignment**

Now the query actually finishes; however, it is still much slower than before. There are more changes that can be made to the query to make it run faster. Your assignment (and challenge!) is to adjust the query so that it runs in similar speed to the query with optimizations enabled. You will be the human query optimizer replacing the disabled one.


In [ ]:
query = """
SELECT
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) AS revenue,
    o_orderdate,
    o_shippriority
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
    AND o_orderdate < CAST('1995-03-15' AS date)
    AND l_shipdate > CAST('1995-03-15' AS date)
GROUP BY
    l_orderkey,
    o_orderdate,
    o_shippriority
ORDER BY
    revenue DESC,
    o_orderdate
LIMIT 10;
"""

run_and_profile_query(query)


# **Bonus Assignment 1**

The TPC-H queries can be loaded from DuckDB using the query `SELECT * FROM tpch_queries()`. Run all the queries in both DuckDB and SQLite and compare the results.

Note: Not all queries will work as-is in SQLite, and some might need to be (slightly) rewritten to accomodate SQLite's (more limited) SQL dialect.

# **Bonus Assignment 2**

As a bonus assignment, here is another query that you can optimize.

In [ ]:
query = """
SELECT
    nation,
    o_year,
    sum(amount) AS sum_profit
FROM (
    SELECT
        n_name AS nation,
        extract(year FROM o_orderdate) AS o_year,
        l_extendedprice * (1 - l_discount) - ps_supplycost * l_quantity AS amount
    FROM
        part,
        supplier,
        lineitem,
        partsupp,
        orders,
        nation
    WHERE
        s_suppkey = l_suppkey
        AND ps_suppkey = l_suppkey
        AND ps_partkey = l_partkey
        AND p_partkey = l_partkey
        AND o_orderkey = l_orderkey
        AND s_nationkey = n_nationkey
        AND p_name LIKE '%green%') AS profit
GROUP BY
    nation,
    o_year
ORDER BY
    nation,
    o_year DESC;
"""


run_and_profile_query(query)

## Experiments with my own data

In [3]:
!pip install duckdb --pre

     |████████████████████████████████| 14.0 MB 5.5 MB/s 


In [9]:
file_path = r"/content/filings_13f_xml_flag.csv"

import duckdb
conn_duck = duckdb.connect("xml_duck",read_only=False)

import sqlite3
conn_sqlite = sqlite3.connect("xml_sqlite.db")


In [10]:
import pandas as pd
df = pd.read_csv(file_path)
df.shape

conn_duck.execute("CREATE TABLE xml_duck AS SELECT * FROM df")
df.to_sql("xml_sqlite", conn_sqlite, if_exists="replace")



RuntimeError: ignored

In [ ]:
conn_duck.execute("SHOW TABLES").df()
conn_duck.execute("SELECT * FROM xml_duck LIMIT 10").df()

In [12]:
cursor = conn_sqlite.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('xml_sqlite',)]


In [92]:
conn_sqlite.execute("SELECT managerName FROM xml_sqlite LIMIT 10").fetchall()

[('STRATEGIC INVESTMENT MANAGEMENT',),
 ('NATIONWIDE LIFE INSURANCE CO /OH',),
 ('NATIONWIDE LIFE INSURANCE CO /OH',),
 ('NATIONWIDE LIFE INSURANCE CO /OH',),
 ('NATIONWIDE LIFE INSURANCE CO /OH',),
 ('NATIONWIDE ADVISORY SERVICES INC/NEW',),
 ('NATIONWIDE ADVISORY SERVICES INC/NEW',),
 ('NATIONWIDE ADVISORY SERVICES INC/NEW',),
 ('NATIONWIDE CORP',),
 ('CALIFORNIA CASUALTY MANAGEMENT CO',)]

In [134]:
# Performance of SQLite on xml_flag table

%%time
conn_sqlite.execute("SELECT managerName, COUNT (DISTINCT filedAsOfDate),\
COUNT(DISTINCT created_at),\
SUM(report_Year), \
AVG(report_Year), \
COUNT (DISTINCT accessionNumber) \
FROM xml_sqlite GROUP BY managerName, periodOfReport, accessionNumber").fetchall()

"""
CPU times: user 132 ms, sys: 23.1 ms, total: 155 ms
Wall time: 160 ms
"""

CPU times: user 1.57 s, sys: 53.2 ms, total: 1.62 s
Wall time: 1.63 s


In [124]:
conn_duck.execute("PRAGMA enable_optimizer")

In [136]:
# Performance of DuckDB on xml_flag table

%%time

conn_duck.execute("SELECT managerName, COUNT (DISTINCT filedAsOfDate), \
COUNT(DISTINCT created_at),\
SUM(report_Year), \
AVG(report_Year), \
COUNT (DISTINCT accessionNumber) \
FROM xml_duck GROUP BY managerName, periodOfReport, accessionNumber").fetchnumpy()

"""
CPU times: user 55.7 ms, sys: 15 ms, total: 70.7 ms
Wall time: 71.2 ms
[(4833,)]
"""

CPU times: user 248 ms, sys: 7.08 ms, total: 255 ms
Wall time: 253 ms


### Duckdb is constantly quicker